# Gilead - List
## Info
* Summe falsch angeliefert

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("Gilead_Sciences_Switzerland_EFPIA_Report_2017.pdf", pages='all', lattice=True, pandas_options={'thousands': '.'})

## Format Table

In [3]:
df_export = df.copy()

write_to_excel(df_export, 'tmp.xlsx')

#Rename Columns
df_export.columns = ['empty', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Shift row
df_export.loc[df_export['empty'].str.len() < 2, 'empty'] = np.NaN
df_export[df_export['empty'].notna()] = df_export[df_export['empty'].notna()].shift(1, axis='columns')

#Remove empty rows
df_export = df_export.dropna(subset=['total'], how='all')

#Remove empty columns
df_export.drop(columns=['empty', 'empty1'], inplace=True)

#Remove Carination
df_export = remove_carination(df_export, " ")

#Remove Title
df_export['name'] = df_export['name'].str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export['name'].str.replace('Unknown ', '')
df_export['name'] = df_export['name'].str.replace('Praktischer Arzt/in', '')
df_export['name'] = df_export['name'].str.strip()

#Add Type
df_export = set_type_by_alphabetical_order(df_export)

#Revert name
df_export = revert_name(df_export, ' ')

#Add PLZ
df_export = add_plz(df_export)

#Convert to Numbers manually. Cannot use standard functions, because "total" is already a number
df_export['donations_grants'] = df_export['donations_grants'].str.replace(".", "")
df_export['sponsorship'] = df_export['sponsorship'].str.replace(".", "")
df_export['registration_fees'] = df_export['registration_fees'].str.replace(".", "")
df_export['travel_accommodation'] = df_export['travel_accommodation'].str.replace(".", "")
df_export['fees'] = df_export['fees'].str.replace(".", "")
df_export['related_expenses'] = df_export['related_expenses'].str.replace(".", "")

df_export = amounts_to_number(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'gilead')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')